In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from collections import Counter
import nltk
from nltk.corpus import stopwords
import pickle
from scipy.sparse import load_npz
import os

# Побудова та порівняння моделей

## From Baseline Evaluation to Full Model Comparison

У попередньому кроці ми навчали **одну базову модель** (Logistic Regression) та оцінювали її на **валідаційній вибірці**.  
Мета була: **отримати початкову оцінку** та перевірити pipeline для подальших експериментів.

У цьому кроці ми будуємо **повноцінний workflow для порівняння моделей**:

- Навчаємо і оцінюємо **три моделі**: Logistic Regression, Random Forest, Multinomial Naive Bayes
- Можливе **налаштування гіперпараметрів** (GridSearchCV)
- Порівнюємо моделі за **F1_macro** (primary metric) та Accuracy (secondary metric)
- Створюємо **зручну таблицю результатів**, щоб вибрати найкращу модель для фінальної оцінки на тестовій вибірці

### Основна різниця між кроками:

| Крок                     | Мета                                | Модель               | Особливості |
|---------------------------|------------------------------------|--------------------|-------------|
| Baseline / Evaluation     | Перевірити pipeline, початкові метрики | 1 базова модель     | Без порівняння, без GridSearchCV |
| Model Training & Comparison | Порівняти кілька моделей, підібрати найкращу | 3 моделі + tuning  | Порівняння, hyperparameter tuning, таблиця результатів |


## Model Training and Comparison

У цій частині ми будуємо повний **workflow для порівняння моделей** з трьома різними моделями, налаштуванням гіперпараметрів та оцінкою на **валідаційній та тестовій вибірках**.  
Також створюємо **таблицю результатів** для наочного порівняння.

### Використовувані моделі:

- **Logistic Regression** (TF-IDF + табличні ознаки)  
- **Random Forest Classifier** (TF-IDF + табличні ознаки)  
- **Multinomial Naive Bayes** (тільки TF-IDF)

### Підхід:

- Для налаштування гіперпараметрів використовуємо **GridSearchCV**  
- Фіксуємо **train/validation scores**, час навчання та обрані параметри  
- Оцінюємо результати за **F1_macro** (primary metric) та Accuracy (secondary metric)  
- Створюємо зручну **таблицю порівняння моделей**, щоб легко обрати найкращу


In [3]:
import pickle
from scipy.sparse import load_npz
from scipy.sparse import hstack
import os

data_dir = "/content/"  # шлях до твоїх збережених даних

# Табличні ознаки
with open(os.path.join(data_dir, "X_tab_train.pkl"), "rb") as f:
    X_tab_train = pickle.load(f)
with open(os.path.join(data_dir, "X_tab_val.pkl"), "rb") as f:
    X_tab_val = pickle.load(f)
with open(os.path.join(data_dir, "X_tab_test.pkl"), "rb") as f:
    X_tab_test = pickle.load(f)

# Текстові ознаки (розріджена матриця)
X_text_train = load_npz(os.path.join(data_dir, "X_text_train.npz"))
X_text_val = load_npz(os.path.join(data_dir, "X_text_val.npz"))
X_text_test = load_npz(os.path.join(data_dir, "X_text_test.npz"))

# Мітки
with open(os.path.join(data_dir, "y_train.pkl"), "rb") as f:
    y_train = pickle.load(f)
with open(os.path.join(data_dir, "y_val.pkl"), "rb") as f:
    y_val = pickle.load(f)
with open(os.path.join(data_dir, "y_test.pkl"), "rb") as f:
    y_test = pickle.load(f)


# Prepare combined features

X_train_combined = hstack([X_tab_train, X_text_train])
X_val_combined = hstack([X_tab_val, X_text_val])
X_test_combined = hstack([X_tab_test, X_text_test])

print("Data loaded successfully!")
print("Train combined shape:", X_train_combined.shape)
print("Validation combined shape:", X_val_combined.shape)
print("Test combined shape:", X_test_combined.shape)

Data loaded successfully!
Train combined shape: (1786, 5005)
Validation combined shape: (316, 5005)
Test combined shape: (371, 5005)


In [4]:
import time
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score, accuracy_score
from scipy.sparse import hstack


# 1. Prepare combined features

# Already done:
# X_train_combined, X_val_combined, X_test_combined
# X_text_train, X_text_val, X_text_test (TF-IDF only)
# y_train, y_val, y_test

# Combine tabular and text features for the test set
# This line is redundant as X_test_combined is already created in the previous cell
# X_test_combined = hstack([X_tab_test, X_text_test])


# 2.  Model configurations for tuning

models_params = {
    "Logistic Regression": {
        "model": LogisticRegression(class_weight="balanced", max_iter=500, random_state=42),
        "params": {"C": [0.1, 1, 5, 10], "solver": ["lbfgs", "liblinear"]}
    },
    "Random Forest": {
        "model": RandomForestClassifier(class_weight="balanced", random_state=42),
        "params": {"n_estimators": [100, 200], "max_depth": [None, 10, 20]}
    },
    "Multinomial NB": {
        "model": MultinomialNB(),
        "params": {"alpha": [0.5, 1.0, 2.0]}
    }
}


# 3. Run GridSearchCV, evaluate
results = []

for name, mp in models_params.items():
    print(f"\n {name} ")
    start_time = time.time()

    model = mp["model"]
    param_grid = mp["params"]

    # Choose features
    if name == "Multinomial NB":
        X_train_feat = X_text_train
        X_val_feat = X_text_val
        X_test_feat = X_text_test
    else:
        X_train_feat = X_train_combined
        X_val_feat = X_val_combined
        X_test_feat = X_test_combined

    # GridSearchCV
    gs = GridSearchCV(model, param_grid, scoring="f1_macro", cv=3, n_jobs=-1)
    gs.fit(X_train_feat, y_train)

    best_model = gs.best_estimator_

    # Evaluate
    y_train_pred = best_model.predict(X_train_feat)
    y_val_pred = best_model.predict(X_val_feat)
    y_test_pred = best_model.predict(X_test_feat)

    f1_train = f1_score(y_train, y_train_pred, average="macro")
    f1_val = f1_score(y_val, y_val_pred, average="macro")
    f1_test = f1_score(y_test, y_test_pred, average="macro")

    elapsed_time = round(time.time() - start_time, 2)

    print(f"Best params: {gs.best_params_}")
    print(f"F1 macro train: {round(f1_train,3)} | val: {round(f1_val,3)} | test: {round(f1_test,3)}")
    print(f"Time: {elapsed_time} sec")

    results.append({
        "Model": name,
        "Parameters": gs.best_params_,
        "Train F1_macro": round(f1_train,3),
        "Validation F1_macro": round(f1_val,3),
        "Test F1_macro": round(f1_test,3),
        "Training Time (sec)": elapsed_time
    })


# 4. Results table

results_df = pd.DataFrame(results).sort_values(by="Validation F1_macro", ascending=False)
print("\n=== Model Comparison Table ===")
display(results_df)


 Logistic Regression 


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Best params: {'C': 10, 'solver': 'lbfgs'}
F1 macro train: 0.976 | val: 0.59 | test: 0.585
Time: 37.7 sec

 Random Forest 
Best params: {'max_depth': None, 'n_estimators': 200}
F1 macro train: 1.0 | val: 0.564 | test: 0.562
Time: 21.71 sec

 Multinomial NB 
Best params: {'alpha': 0.5}
F1 macro train: 0.636 | val: 0.596 | test: 0.617
Time: 0.07 sec

=== Model Comparison Table ===


,Model,Parameters,Train F1_macro,Validation F1_macro,Test F1_macro,Training Time (sec)
2,Multinomial NB,{'alpha': 0.5},0.636,0.596,0.617,0.07
0,Logistic Regression,"{'C': 10, 'solver': 'lbfgs'}",0.976,0.590,0.585,37.70
1,Random Forest,"{'max_depth': None, 'n_estimators': 200}",1.000,0.564,0.562,21.71


## Порівняння моделей

Ми порівняли три моделі на train, validation та test вибірках за F1_macro та часом навчання.

### Результати

| Модель                | F1_macro (Train) | F1_macro (Validation) | F1_macro (Test) | Час навчання (сек) |
|----------------------|-----------------|---------------------|----------------|------------------|
| Multinomial NB       | 0.636           | 0.596               | 0.617          | 0.07             |
| Logistic Regression  | 0.976           | 0.590               | 0.585          | 37.70            |
| Random Forest        | 1.000           | 0.564               | 0.562          | 21.71            |

---

### Короткі висновки

- **Logistic Regression**: дуже добре запам’ятала тренувальні дані (overfitting), але не дуже добре працює на нових.  
- **Random Forest**: схожа ситуація, трохи гірше на validation.  
- **Multinomial NB**: менше overfitting, найкраща узагальнюваність, швидка.  

---

### Спостереження

- LR та RF переобучені → F1 на train дуже високий, але на validation нижчий.  
- NB має більш стабільні результати → краща для текстових ознак.  
- **Рекомендована модель для подальшого використання:** Multinomial NB.

---

### Можливі покращення для LR / RF

- Зменшити розмірність TF-IDF  
- Використовувати регуляризацію  
- Спробувати ансамблеві або гібридні моделі  
- Для LR: збільшити `max_iter` і масштабувати ознаки
